In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 18 11:55:32 2017

Script for full tests, decision tree (pruned)

"""

import sklearn.model_selection as ms
import pandas as pd
from helpers import basicResults,dtclf_pruned,makeTimingCurve
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

def DTpruningVSnodes(clf,alphas,trgX,trgY,dataset):
    '''Dump table of pruning alpha vs. # of internal nodes'''
    out = {}
    for a in alphas:
        clf.set_params(**{'DT__alpha':a})
        clf.fit(trgX,trgY)
        out[a]=clf.steps[-1][-1].numNodes()
        print(dataset,a)
    out = pd.Series(out)
    out.index.name='alpha'
    out.name = 'Number of Internal Nodes'
    out.to_csv('./output/DT_{}_nodecounts.csv'.format(dataset))
    
    return



    

# Load Data       
cancer = pd.read_csv('./data/breast_cancer.csv')  
cancer = pd.get_dummies(cancer, columns = ['class'], prefix = 'class')
cancer['class'] = cancer['class_2.0']
cancer.drop(['class_2.0', 'class_4.0'], axis = 1, inplace = True)
cancer = cancer.astype(np.int64)

cancerX = cancer.drop('class',1).copy().values
cancerY = cancer['class'].copy().values


contra = pd.read_csv('./data/cmc.data.txt')      
col_names_contra = ['wifes_age', 'wifes_edu', 'husbs_edu', 'num_children_born', 'wifes_religion', 'wife_employed', 'husbs_occup', 'SOL_index', 'media_expose', 'contra_method']
contra.columns = col_names_contra
contra[col_names_contra] = contra[col_names_contra].astype(np.int64)
contra = pd.get_dummies(contra, columns = ['husbs_occup'], prefix = 'husbs_occup')

contraX = contra.drop('contra_method',1).copy().values
contraY = contra['contra_method'].copy().values



contra_trgX, contra_tstX, contra_trgY, contra_tstY = ms.train_test_split(contraX, contraY, test_size=0.3, random_state=0,stratify=contraY)     
cancer_trgX, cancer_tstX, cancer_trgY, cancer_tstY = ms.train_test_split(cancerX, cancerY, test_size=0.3, random_state=0,stratify=cancerY)     

# Search for good alphas
alphas = [-1,-1e-3,-(1e-3)*10**-0.5, -1e-2, -(1e-2)*10**-0.5,-1e-1,-(1e-1)*10**-0.5, 0, (1e-1)*10**-0.5,1e-1,(1e-2)*10**-0.5,1e-2,(1e-3)*10**-0.5,1e-3]


pipeO = Pipeline([('Scale',StandardScaler()),                 
                 ('DT',dtclf_pruned(random_state=55))])
pipeC = Pipeline([('Scale',StandardScaler()),                 
                 ('DT',dtclf_pruned(random_state=55))])

params = {'DT__criterion':['gini','entropy'],'DT__alpha':alphas,'DT__class_weight':['balanced']}

cancer_clf = basicResults(pipeC,cancer_trgX,cancer_trgY,cancer_tstX,cancer_tstY,params,'DT','cancer')        
contra_clf = basicResults(pipeO,contra_trgX,contra_trgY,contra_tstX,contra_tstY,params,'DT','contra')        


cancer_final_params = cancer_clf.best_params_
contra_final_params = contra_clf.best_params_

pipeC.set_params(**cancer_final_params)
makeTimingCurve(cancerX,cancerY,pipeC,'DT','cancer')
pipeO.set_params(**contra_final_params)
makeTimingCurve(contraX,contraY,pipeO,'DT','contra')


DTpruningVSnodes(pipeC,alphas,cancer_trgX,cancer_trgY,'cancer')
DTpruningVSnodes(pipeO,alphas,contra_trgX,contra_trgY,'contra')

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini .....
[CV]  DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini, score=0.8455313159746661, total=   0.0s
[CV] DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini .....
[CV]  DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini, score=0.9035890218156231, total=   0.0s
[CV] DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini .....
[CV]  DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini, score=0.9208304011259677, total=   0.0s
[CV] DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini .....
[CV]  DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini, score=0.9066091954022988, total=   0.0s
[CV] DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini .....
[CV]  DT__alpha=-1, DT__class_weight=balanced, DT__criterion=gini, score=0.8973214285714284, total=   0.0s
[CV] DT__alpha=-1, DT__class_weight=balanced, DT

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:

[CV]  DT__alpha=-0.01, DT__class_weight=balanced, DT__criterion=entropy, score=0.9238505747126436, total=   0.0s
[CV] DT__alpha=-0.01, DT__class_weight=balanced, DT__criterion=entropy 
[CV]  DT__alpha=-0.01, DT__class_weight=balanced, DT__criterion=entropy, score=0.9583333333333333, total=   0.0s
[CV] DT__alpha=-0.0031622776601683794, DT__class_weight=balanced, DT__criterion=gini 
[CV]  DT__alpha=-0.0031622776601683794, DT__class_weight=balanced, DT__criterion=gini, score=0.8455313159746661, total=   0.0s
[CV] DT__alpha=-0.0031622776601683794, DT__class_weight=balanced, DT__criterion=gini 
[CV]  DT__alpha=-0.0031622776601683794, DT__class_weight=balanced, DT__criterion=gini, score=0.9725545390570022, total=   0.0s
[CV] DT__alpha=-0.0031622776601683794, DT__class_weight=balanced, DT__criterion=gini 
[CV]  DT__alpha=-0.0031622776601683794, DT__class_weight=balanced, DT__criterion=gini, score=0.9553131597466572, total=   0.0s
[CV] DT__alpha=-0.0031622776601683794, DT__class_weight=balance

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConve

[CV]  DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini, score=0.989795918367347, total=   0.0s
[CV] DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini ......
[CV]  DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini, score=0.9451090781140044, total=   0.0s
[CV] DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini ......
[CV]  DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini, score=0.9238505747126436, total=   0.0s
[CV] DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini ......
[CV]  DT__alpha=0, DT__class_weight=balanced, DT__criterion=gini, score=0.9508928571428571, total=   0.0s
[CV] DT__alpha=0, DT__class_weight=balanced, DT__criterion=entropy ...
[CV]  DT__alpha=0, DT__class_weight=balanced, DT__criterion=entropy, score=0.8416608022519356, total=   0.0s
[CV] DT__alpha=0, DT__class_weight=balanced, DT__criterion=entropy ...
[CV]  DT__alpha=0, DT__class_weight=balanced, DT__criterion=entropy, score=0.9521463757916961, total=   0.0s
[CV]

...

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


contra 0.1
contra 0.0031622776601683794
contra 0.01


/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


contra 0.00031622776601683794
contra 0.001


/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
